In [2]:
import scraping_utils as sc
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
content = sc.get_content("http://books.toscrape.com/index.html") #scrape the home to get the list of the genres and their links
soup = BeautifulSoup(content, "html.parser")

In [5]:
genres = soup.find("ul", {"class":"nav nav-list"}).find("li").find("ul").find_all("a")
genres_labels = [] #stores the names of the genres
genres_links = [] #stores the links of the different genres
for genre in genres:
    genres_labels.append(genre.text.replace("\n","").strip().lower())
    genres_links.append(genre["href"])

In [6]:
selected_genre=input("Enter Genre: ").lower()
if selected_genre == "All" or selected_genre == "Books":
    genre_url="https://books.toscrape.com/catalogue/category/books_1/index.html"
else:
    genre_url="https://books.toscrape.com/"+genres_links[genres_labels.index(selected_genre)]
content = sc.get_content(genre_url)
soup = BeautifulSoup(content, "html.parser")

In [7]:
products = soup.find_all("article", {"class":"product_pod"})
l=[]
for product in products:
    d={}
    d["Name"]=product.find("h3").find("a")["title"]
    d["Price"]=product.find("p", {"class":"price_color"}).text
    d["Avalibility"]=product.find("p", {"class":"instock availability"}).text.replace("\n","").strip()
    if product.find("p", {"class":"star-rating One"}) != None:
        d["Rating"]=1
    elif product.find("p", {"class":"star-rating Two"}) != None:
        d["Rating"]=2
    elif product.find("p", {"class":"star-rating Three"}) != None:
        d["Rating"]=3
    elif product.find("p", {"class":"star-rating Four"}) != None:
        d["Rating"]=4
    else:
        d["Rating"]=5
    l.append(d)

In [8]:
res_count = soup.find("form", {"class":"form-horizontal"}).find_all("strong")
if len(res_count) > 1:
    page_count=2
    base_url=genre_url[0:-10]+"page-"
    while True:
        content=sc.get_content(base_url,str(page_count),".html")
        soup = BeautifulSoup(content, "html.parser")
        products = soup.find_all("article", {"class":"product_pod"})
        for product in products:
            d={}
            d["Name"]=product.find("h3").find("a")["title"]
            d["Price"]=product.find("p", "price_color").text
            d["Avalibility"]=product.find("p", "instock availability").text.replace("\n","").strip()
            if product.find("p", {"class":"star-rating One"}) != None:
                d["Rating"]=1
            elif product.find("p", {"class":"star-rating Two"}) != None:
                d["Rating"]=2
            elif product.find("p", {"class":"star-rating Three"}) != None:
                d["Rating"]=3
            elif product.find("p", {"class":"star-rating Four"}) != None:
                d["Rating"]=4
            else:
                d["Rating"]=5
            l.append(d)
        res_count = soup.find("form", {"class":"form-horizontal"}).find_all("strong")
        page_count += 1
        if int(res_count[0].text) == int(res_count[2].text):
            break

In [9]:
df=pd.DataFrame(l)
sc.save_data_csv(df, "scraped_data", selected_genre)